In [1]:
# The contents of this files are based on the tensor flow data creation tutorial
# https://www.tensorflow.org/tutorials/load_data/tfrecord

import tensorflow as tf
from level.LevelReader import LevelReader
from pathlib import Path


In [2]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
def image_example(image_string, label):
  image_shape = tf.io.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

In [5]:
from level.LevelVisualizer import LevelVisualizer
from level.Level import Level
from GameManagement.GameConnection import GameConnection
import os
from util.Config import Config
from GameManagement.GameManager import GameManager
from util import ProgramArguments

parser = ProgramArguments.get_program_arguments()
config = parser.parse_args()
config.game_folder_path = os.path.normpath('../science_birds/{os}')

config = Config(config)
game_connection = GameConnection(conf = config)
game_manager = GameManager(conf = config, game_connection = game_connection)
level_reader = LevelReader()

game_manager.start_game()
data_dict = dict()

for original_data_level in sorted(Path("train/generated/").glob('*.xml')):
    parsed_level = level_reader.parse_level(str(original_data_level), blocks = True, pigs = True, platform = True)
    parsed_level.filter_slingshot_platform()

    parsed_level.normalize()
    parsed_level.create_polygons()
    level_structures = parsed_level.separate_structures()
    level_visualizer = LevelVisualizer()

    level_counter = 0
    for idx, structure in enumerate(level_structures):
        meta_data = Level.calc_structure_meta_data(structure)
        struct_doc = level_reader.create_level_from_structure(
            structure = structure,
            level = parsed_level,
            move_to_ground = True
        )
        new_level_path = f'../data/train/structures/level-0{level_counter + 4}.xml'
        level_reader.write_xml_file(struct_doc, new_level_path)
        game_manager.change_level(path = new_level_path)

        new_level = level_reader.parse_level(new_level_path, use_platform = True)
        new_level.normalize()
        new_level.create_polygons()
        new_level.filter_slingshot_platform()
        new_level.separate_structures()

        game_connection.create_level_img(structure = True)
        game_connection.startAi(start_level = level_counter, end_level = level_counter)
        ret_pictures = new_level.create_img(per_structure = True, dot_version = True)
        game_connection.wait_till_all_level_played()
        game_connection.stopAI()
        new_level_data = game_connection.get_data()
        data_dict[f'{original_data_level}_{level_counter}'] = dict(
            meta_data = meta_data,
            game_data = new_level_data,
            img_data = ret_pictures
        )
        level_counter += 1

usage: ipykernel_launcher.py [-h] [--generator GENERATOR]
                             [--level_amount LEVEL_AMOUNT]
                             [--level_path LEVEL_PATH]
                             [--game_folder_path GAME_FOLDER_PATH]
                             [--ai_path AI_PATH] [--rescue_level RESCUE_LEVEL]
                             [--evaluate EVALUATE]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\Blaxzter\AppData\Roaming\jupyter\runtime\kernel-b1ddef90-589f-4b7b-a858-de8faee9b9c8.json


SystemExit: 2

In [ ]:
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())